In [2]:
%%writefile ../NGSEP_pipeline_full.py
#! /usr/bin/env python
### Script for aligning reads
import argparse
import glob
from os.path import basename
from subprocess import call

parser=argparse.ArgumentParser(prog='NGSEP_pipeline_full', description='Script for align, postprocess and call reads using entirely NGSEP')
parser.add_argument('-i', '--input', dest='fastqreads', help='The folder with your fastq reads')
parser.add_argument('-r', '--referencegenome', dest='ref',
                    help='The reference genome file')
#parser.add_argument('-o', '--output', dest='outfolder', help='The output folder')
arg=parser.parse_args()

## Check if there is some parameter missing
if 'None' in str(arg):
        parser.error('Input parameter missing!! Please check your command line parameters with -h or --help')

## sample params
fastq=arg.fastqreads.split('/')[0] # just for avoid to duplication of the '/' at the end
ref=arg.ref
reads=glob.glob('%s/*' % fastq)

call('mkdir -p aln_res', shell=True)
# java -jar NGSEPcore_<VERSION>.jar ReadsAligner -r <REF.fa> -i <SMPL>.fastq -s <SMPL> -o <SMPL>.bam > <SMPL>_aln.log
# java -jar picard.jar SortSam SO=coordinate CREATE_INDEX=true I=<SMPL>.bam O=<SMPL>_sorted.bam >& <SMPL>_sort.log
for i in reads:
    smpl = basename(i).split('.')[0]
    print('Start aligning sample %s' % smpl)
    bam_out= smpl +'_aln.bam'
    aln_cmd='''java -jar src/NGSEPcore_4.1.0.jar ReadsAligner -r %s -i %s -s %s -o aln_res/%s > logs/%s_aln.log''' % (ref, i, smpl, bam_out, smpl)
    print(aln_cmd)
    call(aln_cmd, shell=True)
    picard_cmd = '''java -jar src/picard.jar SortSam SO=coordinate CREATE_INDEX=true I=aln_res/%s O=aln_res/%s_sorted.bam >& logs/%s_sort.log''' % (bam_out, smpl, smpl)
    print(picard_cmd)
    call(picard_cmd, shell=True)
    

# Multisamples SNPs calling

#java -jar src/NGSEPcore_4.1.0.jar MultisampleVariantsDetector -maxBaseQS 30 -maxAlnsPerStartPos 100 -r <REF.fa> -o population.vcf <BAM_FILES>* >& population.log
snp_allign_cmd = '''java -jar src/NGSEPcore_4.1.0.jar MultisampleVariantsDetector -maxBaseQS 30 -maxAlnsPerStartPos 100 -r %s -o FINAL_SNP_file.vcf res_aln/*_sorted.bam >& logs/population.log''' % ref
print(snp_allign_cmd)
call(snp_allign_cmd, shell=True)

Overwriting ../NGSEP_pipeline_full.py


Make script of script

In [2]:
%%writefile ../NGSEP_aln_slurm.py
import os
import argparse
import glob
from os.path import basename

parser=argparse.ArgumentParser(prog='NGSEP_aln_slurm', description='Script for creating slurm jobs for aligning and postprocessing reads using entirely NGSEP')
parser.add_argument('-i', '--input', dest='fastqreads', help='The folder with your fastq reads')
parser.add_argument('-r', '--referencegenome', dest='ref',
                    help='The reference genome file')
#parser.add_argument('-o', '--output', dest='outfolder', help='The output folder')
arg=parser.parse_args()

## Check if there is some parameter missing
if 'None' in str(arg):
        parser.error('Input parameter missing!! Please check your command line parameters with -h or --help')

## sample params
fastq=arg.fastqreads.split('/')[0] # just for avoid to duplication of the '/' at the end
ref=arg.ref
reads=glob.glob('%s/*' % fastq)

slurm_header = '''#!/bin/bash
#SBATCH --time=24:00:00
#SBATCH --mem=64gb
#SBATCH --job-name=%s
#SBATCH --error=/work/agro932/sybarreral/agrobinf/sybarreral/log/%s.err
#SBATCH --output=/work/agro932/sybarreral/agrobinf/sybarreral/log/%s.out

module load java/12
module load python/3.8


'''

for i in reads:
    sample = basename(i).split('.')[0]
    slurm = slurm_header%(sample, sample, sample)
    bam_out= sample +'_aln.bam'
    aln_cmd='''java -Xmx64g -jar src/NGSEPcore_4.1.0.jar ReadsAligner -r %s -i %s -s %s -o aln_res/%s > logs/%s_aln.log\n''' % (ref, i, sample, bam_out, sample)
    slurm += aln_cmd
    picard_cmd = '''java -Xmx64g -jar src/picard.jar SortSam SO=coordinate CREATE_INDEX=true I=aln_res/%s O=aln_res/%s_sorted.bam >& logs/%s_sort.log\n''' % (bam_out, sample, sample)
    slurm += picard_cmd
    with open('NGSEP_align_%s.slurm'%sample, 'w') as f:
                f.write(slurm)

Overwriting ../NGSEP_aln_slurm.py


slurm bowtie 2

In [6]:
%%writefile ../bowtie2_aln_slurm.py
import os
import argparse
import glob
from os.path import basename

parser=argparse.ArgumentParser(prog='bowtie2_aln_slurm', description='Script for creating slurm jobs for aligning reads with bowtie2')
parser.add_argument('-i', '--input', dest='fastqreads', help='The folder with your fastq reads')
arg=parser.parse_args()

## Check if there is some parameter missing
if 'None' in str(arg):
        parser.error('Input parameter missing!! Please check your command line parameters with -h or --help')

## sample params
fastq=arg.fastqreads.split('/')[0] # just for avoid to duplication of the '/' at the end
reads=glob.glob('%s/*' % fastq)

slurm_header = '''#!/bin/sh
#SBATCH --ntasks-per-node=4
#SBATCH --nodes=1
#SBATCH --mem=56gb
#SBATCH --time=20:00:00
#SBATCH --job-name=BTmap_%s
#SBATCH --mail-user=sybarreral@huskers.unl.edu
#SBATCH --mail-type=ALL
#SBATCH --error=/work/agro932/sybarreral/agrobinf/sybarreral/log/map_bt2_%s.err
#SBATCH --output=/work/agro932/sybarreral/agrobinf/sybarreral/log/map_bt2_%s.out

module load bowtie/2.3

IDXPTH=/work/agro932/sybarreral/agrobinf/sybarreral/project/ucdReads/H2214/NGSEP_varcall_full/REF
READPTH=/work/agro932/sybarreral/agrobinf/sybarreral/project/ucdReads/H2214/NGSEP_varcall_full/FASTQ

'''

for i in reads:
    sample = basename(i).split('.')[0]
    slurm = slurm_header%(sample, sample, sample)
    aln_cmd='''bowtie2 --local -p 4 -N 1 -x $IDXPTH/Pacutifolius -U $READPTH/%s -S aln_res/%s_pha.sam\n''' % (basename(i), sample)
    slurm += aln_cmd
    with open('bowtie2_align_%s.slurm'%sample, 'w') as f:
                f.write(slurm)

Overwriting ../bowtie2_aln_slurm.py


In [7]:
%%writefile ../samtools_sort_slurm.py
import os
import argparse
import glob
from os.path import basename

parser=argparse.ArgumentParser(prog='samtools_sort_slurm', description='Script for creating slurm jobs for samtools sort')
parser.add_argument('-i', '--input', dest='fastqreads', help='The folder with your fastq reads')
arg=parser.parse_args()

## Check if there is some parameter missing
if 'None' in str(arg):
        parser.error('Input parameter missing!! Please check your command line parameters with -h or --help')

## sample params
fastq=arg.fastqreads.split('/')[0] # just for avoid to duplication of the '/' at the end
reads=glob.glob('%s/*' % fastq)

slurm_header = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --mem=64gb
#SBATCH --time=10:00:00
#SBATCH --job-name=sortReads_%s
#SBATCH --mail-user=sybarreral@huskers.unl.edu
#SBATCH --mail-type=ALL
#SBATCH --error=/work/agrobinf/SHARED/agrobinf2020/users/sybarreral/log/sort_%s.err
#SBATCH --output=/work/agrobinf/SHARED/agrobinf2020/users/sybarreral/log/sort_%s.out

module load samtools/1.9

'''

for i in reads:
    sample = basename(i).split('.')[0]
    slurm = slurm_header%(sample, sample, sample)
    aln_cmd='''samtools sort -o aln_res/%s_pha.bam aln_res/%s_pha.sam\n''' % (sample, sample)
    slurm += aln_cmd
    with open('samtools_sort_%s.slurm'%sample, 'w') as f:
                f.write(slurm)

Writing ../samtools_sort_slurm.py
